In [1]:
#!pip install textattack[tensorflow,optional]
#!pip install -U datasets

In [2]:
import os
import datetime

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision
import tensorflow as tf
import transformers
from transformers import CLIPProcessor, CLIPModel

import textattack
from textattack.models.wrappers import HuggingFaceModelWrapper

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC


D:\anaconda3\envs\tf\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 找不到指定的程序。'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
D:\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._p

In [3]:
from textattack import Attack
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.constraints.pre_transformation import (
    InputColumnModification,
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.transformations import WordSwapEmbedding, WordSwapWordNet 

from textattack.transformations.composite_transformation import CompositeTransformation

from textattack.attack_recipes.attack_recipe import AttackRecipe


class TFLepsilon(AttackRecipe):

    @staticmethod
    def build(model_wrapper, epsilon=0.4):
        #
        # Swap words with their 50 closest embedding nearest-neighbors.
        # Embedding: Counter-fitted PARAGRAM-SL999 vectors.
        
        ## transformation = CompositeTransformation([WordSwapWordNet(), WordSwapEmbedding(max_candidates=50)])
        transformation = WordSwapEmbedding(max_candidates=50)
        #
        # Don't modify the same word twice or the stopwords defined
        # in the TextFooler public implementation.
        #
        # fmt: off
        stopwords = set(
            ["a", "about", "above", "across", "after", "afterwards", "again", "against", "ain", "all", 
             "almost", "alone", "along", "already", "also", "although", "am", "among", "amongst", "an", 
             "and", "another", "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are", "aren",
             "aren't", "around", "as", "at", "back", "been", "before", "beforehand", "behind", "being", 
             "below", "beside", "besides", "between", "beyond", "both", "but", "by", "can", "cannot", 
             "could", "couldn", "couldn't", "d", "didn", "didn't", "doesn", "doesn't", "don", "don't", 
             "down", "due", "during", "either", "else", "elsewhere", "empty", "enough", "even", "ever", 
             "everyone", "everything", "everywhere", "except", "first", "for", "former", "formerly",
             "from", "hadn", "hadn't", "hasn", "hasn't", "haven", "haven't", "he", "hence", "her", "here", 
             "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", 
             "however", "hundred", "i", "if", "in", "indeed", "into", "is", "isn", "isn't", "it", "it's", 
             "its", "itself", "just", "latter", "latterly", "least", "ll", "may", "me", "meanwhile", 
             "mightn", "mightn't", "mine", "more", "moreover", "most", "mostly", "must", "mustn", "mustn't", 
             "my", "myself", "namely", "needn", "needn't", "neither", "never", "nevertheless", "next", "no",
             "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "o", "of", "off", "on",
             "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", 
             "out", "over", "per", "please", "s", "same", "shan", "shan't", "she", "she's", "should've", 
             "shouldn", "shouldn't", "somehow", "something", "sometime", "somewhere", "such", "t", "than", 
             "that", "that'll", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", 
             "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "this", "those",
             "through", "throughout", "thru", "thus", "to", "too", "toward", "towards", "under", "unless",
             "until", "up", "upon", "used", "ve", "was", "wasn", "wasn't", "we", "were", "weren", "weren't",
             "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", 
             "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", 
             "whole", "whom", "whose", "why", "with", "within", "without", "won", "won't", "would", "wouldn",
             "wouldn't", "y", "yet", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself",
             "yourselves"]
        )
        # fmt: on
        constraints = [RepeatModification(), StopwordModification(stopwords=stopwords)]
        #
        # During entailment, we should only edit the hypothesis - keep the premise
        # the same.
        #
        input_column_modification = InputColumnModification(
            ["premise", "hypothesis"], {"premise"}
        )
        constraints.append(input_column_modification)
        # Minimum word embedding cosine similarity of 0.5.
        # (The paper claims 0.7, but analysis of the released code and some empirical
        # results show that it's 0.5.)
        #
        constraints.append(WordEmbeddingDistance(min_cos_sim=0.5))
        #
        # Only replace words with the same part of speech (or nouns with verbs)
        #
        constraints.append(PartOfSpeech(allow_verb_noun_swap=True))
        #
        # Universal Sentence Encoder with a minimum angular similarity of ε = 0.5.
        #
        # In the TextFooler code, they forget to divide the angle between the two
        # embeddings by pi. So if the original threshold was that 1 - sim >= 0.5, the
        # new threshold is 1 - (0.5) / pi = 0.840845057
        #
        use_constraint = UniversalSentenceEncoder(

            # 1- 0.4/pi = 0.8726760455264837
            threshold = 1-epsilon/np.pi,
            metric="angular",
            compare_against_original=False,
            window_size=15,
            skip_text_shorter_than_window=True,
        )
        constraints.append(use_constraint)
        #
        # Goal is untargeted classification
        #
        goal_function = UntargetedClassification(model_wrapper)
        #
        # Greedily swap words with "Word Importance Ranking".
        #
        search_method = GreedyWordSwapWIR(wir_method="delete")

        return Attack(goal_function, constraints, transformation, search_method)


In [4]:
def runAttack(model_wrapper, n, path, target_ds):
    # used TFLepsilon
    attack = TFLepsilon.build(model_wrapper)
    attack_args = textattack.AttackArgs(num_examples=n, log_to_csv=path, shuffle=True, #parallel=True,
                                        checkpoint_interval=5, checkpoint_dir='checkpoints', 
                                        disable_stdout=True)
    attacker = textattack.Attacker(attack, target_ds, attack_args)
    attacker.attack_dataset()

In [5]:
# classifiers
def adaptation(train_set, test_set):
    lr = LogisticRegression(solver='saga', multi_class='multinomial')
    rf = RandomForestClassifier(max_depth=7)
    knn = KNeighborsClassifier(n_neighbors=7)
    svc = SVC(kernel='linear', probability=True)
    
    trainX = train_set.iloc[:,:-1]
    trainY = train_set.iloc[:,-1]
    testX = test_set.iloc[:,:-1]
    testY = test_set.iloc[:,-1]
    
    for key, clf in {'Log Reg':lr, 'RF':rf, 'KNN':knn, 'SVC':svc}.items():
        clf.fit(trainX, trainY) # fit train data
        pred = clf.predict(testX) # predict test set

        # print accuracy
        print(key, (pred==np.array(testY)).sum()/len(pred))

    return lr, rf, knn, svc

In [6]:
target_ds_mr = textattack.datasets.HuggingFaceDataset('rotten_tomatoes', split='test')

textattack: Loading datasets dataset rotten_tomatoes, split test.


### BERT-based-uncased

In [7]:
# bert-based-uncased
model_path1 = 'bert-base-uncased'

bert_model = transformers.AutoModel.from_pretrained(model_path1)
bert_tokenizer = transformers.AutoTokenizer.from_pretrained(model_path1)

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [8]:
bertmrtrain = pd.read_csv('dataset_embeddings/bertmrtrain.txt', header=None)
bertmrtest = pd.read_csv('dataset_embeddings/bertmrtest.txt', header=None)
lr_bert_mr, rf_bert_mr, knn_bert_mr, svc_bert_mr = adaptation(bertmrtrain, bertmrtest)

D:\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Log Reg 0.8105065666041276
RF 0.7223264540337712
KNN 0.6801125703564728
SVC 0.8030018761726079


In [9]:
# custom model wrapper for CLIP
class BERTModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).pooler_output
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [10]:
bert_model_wrapper_lr_mr = BERTModelWrapper(bert_tokenizer, bert_model, lr_bert_mr)
bert_model_wrapper_rf_mr = BERTModelWrapper(bert_tokenizer, bert_model, rf_bert_mr)
bert_model_wrapper_knn_mr = BERTModelWrapper(bert_tokenizer, bert_model, knn_bert_mr)
bert_model_wrapper_svc_mr = BERTModelWrapper(bert_tokenizer, bert_model, svc_bert_mr)

In [11]:
runAttack(bert_model_wrapper_lr_mr, -1, 'attack_results_new/bertmrattacklr04.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattacklr04.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 0 / 1 / 5:   0%|                         | 5/1066 [00:46<2:45:22,  9.35s/it]textattack: Saving checkpoint under "checkpoints\1720023633912.ta.chkpt" at 2024-07-04 00:20:33 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 0 / 2 / 6:   1%|▏                        | 6/1066 [00:46<2:18:03,  7.81s/it]

[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10:   1%|▏                      | 10/1066 [01:00<1:46:21,  6.04s/it]textattack: Saving checkpoint under "checkpoints\1720023647578.ta.chkpt" at 2024-07-04 00:20:47 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 0 / 4 / 15:   1%|▎                     | 15/1066 [01:44<2:02:24,  6.99s/it]textattack: Saving checkpoint under "checkpoints\1720023691980.ta.chkpt" at 2024-07-04 00:21:31 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 828 / 28 / 199 / 1055:  99%|█████████████▊| 1055/1066 [1:07:50<00:42,  3.86s/it]textattack: Saving checkpoint under "checkpoints\1720027657811.ta.chkpt" at 2024-07-04 01:27:37 after 1055 attacks.


[Succeeded / Failed / Skipped / Total] 832 / 28 / 200 / 1060:  99%|█████████████▉| 1060/1066 [1:08:10<00:23,  3.86s/it]textattack: Saving checkpoint under "checkpoints\1720027677398.ta.chkpt" at 2024-07-04 01:27:57 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 834 / 29 / 202 / 1065: 100%|█████████████▉| 1065/1066 [1:08:42<00:03,  3.87s/it]textattack: Saving checkpoint under "checkpoints\1720027709227.ta.chkpt" at 2024-07-04 01:28:29 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 835 / 29 / 202 / 1066: 100%|██████████████| 1066/1066 [1:08:45<00:00,  3.87s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 835    |
| Number of failed attacks:     | 29     |
| Number of skipped attacks:    | 202    |
| Original accuracy:            | 81.05% |
| Accuracy under attack:        | 2.72%  |
| Attack success rate:          | 96.64% |
| Average perturbed word %:     | 16.15% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 71.08  |
+-------------------------------+--------+


In [12]:
runAttack(bert_model_wrapper_rf_mr, -1, 'attack_results_new/bertmrattackrf04.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattackrf04.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 3 / 0 / 2 / 5:   0%|                         | 5/1066 [00:27<1:37:43,  5.53s/it]textattack: Saving checkpoint under "checkpoints\1720027740516.ta.chkpt" at 2024-07-04 01:29:00 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 0 / 3 / 6:   1%|▏                        | 6/1066 [00:27<1:21:43,  4.63s/it]

[Succeeded / Failed / Skipped / Total] 5 / 0 / 5 / 10:   1%|▏                      | 10/1066 [00:47<1:23:31,  4.75s/it]textattack: Saving checkpoint under "checkpoints\1720027760341.ta.chkpt" at 2024-07-04 01:29:20 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 0 / 6 / 15:   1%|▎                      | 15/1066 [01:07<1:18:35,  4.49s/it]textattack: Saving checkpoint under "checkpoints\1720027780195.ta.chkpt" at 2024-07-04 01:29:40 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 711 / 52 / 292 / 1055:  99%|█████████████▊| 1055/1066 [1:05:08<00:40,  3.70s/it]textattack: Saving checkpoint under "checkpoints\1720031621099.ta.chkpt" at 2024-07-04 02:33:41 after 1055 attacks.


[Succeeded / Failed / Skipped / Total] 715 / 52 / 293 / 1060:  99%|█████████████▉| 1060/1066 [1:05:30<00:22,  3.71s/it]textattack: Saving checkpoint under "checkpoints\1720031642997.ta.chkpt" at 2024-07-04 02:34:02 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 717 / 52 / 296 / 1065: 100%|█████████████▉| 1065/1066 [1:05:34<00:03,  3.69s/it]textattack: Saving checkpoint under "checkpoints\1720031647202.ta.chkpt" at 2024-07-04 02:34:07 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 718 / 52 / 296 / 1066: 100%|██████████████| 1066/1066 [1:05:35<00:00,  3.69s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 718    |
| Number of failed attacks:     | 52     |
| Number of skipped attacks:    | 296    |
| Original accuracy:            | 72.23% |
| Accuracy under attack:        | 4.88%  |
| Attack success rate:          | 93.25% |
| Average perturbed word %:     | 14.72% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 73.77  |
+-------------------------------+--------+


In [13]:
runAttack(bert_model_wrapper_knn_mr, -1, 'attack_results_new/bertmrattackknn04.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattackknn04.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 3 / 0 / 2 / 5:   0%|                         | 5/1066 [01:28<5:11:26, 17.61s/it]textattack: Saving checkpoint under "checkpoints\1720031736292.ta.chkpt" at 2024-07-04 02:35:36 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 0 / 3 / 6:   1%|▏                        | 6/1066 [01:28<4:19:44, 14.70s/it]

[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10:   1%|▏                      | 10/1066 [01:34<2:46:47,  9.48s/it]textattack: Saving checkpoint under "checkpoints\1720031743000.ta.chkpt" at 2024-07-04 02:35:43 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 1 / 6 / 15:   1%|▎                      | 15/1066 [02:49<3:17:27, 11.27s/it]textattack: Saving checkpoint under "checkpoints\1720031817324.ta.chkpt" at 2024-07-04 02:36:57 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 706 / 12 / 337 / 1055:  99%|███████████████▊| 1055/1066 [38:30<00:24,  2.19s/it]textattack: Saving checkpoint under "checkpoints\1720033958347.ta.chkpt" at 2024-07-04 03:12:38 after 1055 attacks.


[Succeeded / Failed / Skipped / Total] 709 / 13 / 338 / 1060:  99%|███████████████▉| 1060/1066 [38:41<00:13,  2.19s/it]textattack: Saving checkpoint under "checkpoints\1720033969976.ta.chkpt" at 2024-07-04 03:12:49 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 712 / 13 / 340 / 1065: 100%|███████████████▉| 1065/1066 [38:49<00:02,  2.19s/it]textattack: Saving checkpoint under "checkpoints\1720033977840.ta.chkpt" at 2024-07-04 03:12:57 after 1065 attacks.
[Succeeded / Failed / Skipped / Total] 712 / 13 / 340 / 1065: 100%|████████████████| 1066/1066 [38:49<00:00,  2.19s/it]

[Succeeded / Failed / Skipped / Total] 712 / 13 / 341 / 1066: 100%|████████████████| 1066/1066 [38:49<00:00,  2.19s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 712    |
| Number of failed attacks:     | 13     |
| Number of skipped attacks:    | 341    |
| Original accuracy:            | 68.01% |
| Accuracy under attack:        | 1.22%  |
| Attack success rate:          | 98.21% |
| Average perturbed word %:     | 9.0%   |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 46.46  |
+-------------------------------+--------+


In [14]:
runAttack(bert_model_wrapper_svc_mr, -1, 'attack_results_new/bertmrattacksvc04.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattacksvc04.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 0 / 1 / 5:   0%|                         | 5/1066 [00:34<2:01:48,  6.89s/it]textattack: Saving checkpoint under "checkpoints\1720034012642.ta.chkpt" at 2024-07-04 03:13:32 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 0 / 2 / 6:   1%|▏                        | 6/1066 [00:34<1:41:52,  5.77s/it]

[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10:   1%|▏                      | 10/1066 [00:47<1:24:28,  4.80s/it]textattack: Saving checkpoint under "checkpoints\1720034026192.ta.chkpt" at 2024-07-04 03:13:46 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 0 / 4 / 15:   1%|▎                     | 15/1066 [01:46<2:04:44,  7.12s/it]textattack: Saving checkpoint under "checkpoints\1720034085024.ta.chkpt" at 2024-07-04 03:14:45 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 816 / 32 / 207 / 1055:  99%|█████████████▊| 1055/1066 [1:08:38<00:42,  3.90s/it]textattack: Saving checkpoint under "checkpoints\1720038096843.ta.chkpt" at 2024-07-04 04:21:36 after 1055 attacks.
[Succeeded / Failed / Skipped / Total] 816 / 32 / 207 / 1055:  99%|█████████████▊| 1056/1066 [1:08:38<00:39,  3.90s/it]

[Succeeded / Failed / Skipped / Total] 820 / 32 / 208 / 1060:  99%|█████████████▉| 1060/1066 [1:09:01<00:23,  3.91s/it]textattack: Saving checkpoint under "checkpoints\1720038119720.ta.chkpt" at 2024-07-04 04:21:59 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 822 / 33 / 210 / 1065: 100%|█████████████▉| 1065/1066 [1:09:30<00:03,  3.92s/it]textattack: Saving checkpoint under "checkpoints\1720038148594.ta.chkpt" at 2024-07-04 04:22:28 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 823 / 33 / 210 / 1066: 100%|██████████████| 1066/1066 [1:09:33<00:00,  3.92s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 823    |
| Number of failed attacks:     | 33     |
| Number of skipped attacks:    | 210    |
| Original accuracy:            | 80.3%  |
| Accuracy under attack:        | 3.1%   |
| Attack success rate:          | 96.14% |
| Average perturbed word %:     | 15.89% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 70.44  |
+-------------------------------+--------+


### DistilBERT-based-uncased

In [15]:
# bert-based-uncased
model_path2 = 'distilbert/distilbert-base-uncased'

disbert_model = transformers.AutoModel.from_pretrained(model_path2)
disbert_tokenizer = transformers.AutoTokenizer.from_pretrained(model_path2)

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [16]:
disbertmrtrain = pd.read_csv('dataset_embeddings/disbertmrtrain.txt', header=None)
disbertmrtest = pd.read_csv('dataset_embeddings/disbertmrtest.txt', header=None)
lr_disbert_mr, rf_disbert_mr, knn_disbert_mr, svc_disbert_mr = adaptation(disbertmrtrain, disbertmrtest)

D:\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Log Reg 0.7945590994371482
RF 0.7682926829268293
KNN 0.7298311444652908
SVC 0.7851782363977486


In [17]:
# custom model wrapper for DistilBERT
class DisBERTModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).last_hidden_state.mean(axis=1)
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [18]:
disbert_model_wrapper_lr_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, lr_disbert_mr)
disbert_model_wrapper_rf_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, rf_disbert_mr)
disbert_model_wrapper_knn_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, knn_disbert_mr)
disbert_model_wrapper_svc_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, svc_disbert_mr)

In [19]:
runAttack(disbert_model_wrapper_lr_mr, -1, 'attack_results_new/disbertmrattacklr04.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattacklr04.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 0 / 1 / 5:   0%|                         | 5/1066 [01:07<4:00:12, 13.58s/it]textattack: Saving checkpoint under "checkpoints\1720038459083.ta.chkpt" at 2024-07-04 04:27:39 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 0 / 3 / 7:   1%|▏                        | 7/1066 [01:08<2:51:43,  9.73s/it]

[Succeeded / Failed / Skipped / Total] 7 / 0 / 3 / 10:   1%|▏                      | 10/1066 [01:14<2:10:40,  7.42s/it]textattack: Saving checkpoint under "checkpoints\1720038465412.ta.chkpt" at 2024-07-04 04:27:45 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 0 / 4 / 15:   1%|▎                     | 15/1066 [01:34<1:50:46,  6.32s/it]textattack: Saving checkpoint under "checkpoints\1720038486022.ta.chkpt" at 2024-07-04 04:28:06 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 814 / 24 / 217 / 1055:  99%|███████████████▊| 1055/1066 [37:07<00:23,  2.11s/it]textattack: Saving checkpoint under "checkpoints\1720040618399.ta.chkpt" at 2024-07-04 05:03:38 after 1055 attacks.
[Succeeded / Failed / Skipped / Total] 814 / 24 / 218 / 1056:  99%|███████████████▊| 1056/1066 [37:07<00:21,  2.11s/it]

[Succeeded / Failed / Skipped / Total] 818 / 24 / 218 / 1060:  99%|███████████████▉| 1060/1066 [37:20<00:12,  2.11s/it]textattack: Saving checkpoint under "checkpoints\1720040631426.ta.chkpt" at 2024-07-04 05:03:51 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 822 / 24 / 219 / 1065: 100%|███████████████▉| 1065/1066 [37:34<00:02,  2.12s/it]textattack: Saving checkpoint under "checkpoints\1720040645820.ta.chkpt" at 2024-07-04 05:04:05 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 823 / 24 / 219 / 1066: 100%|████████████████| 1066/1066 [37:36<00:00,  2.12s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 823    |
| Number of failed attacks:     | 24     |
| Number of skipped attacks:    | 219    |
| Original accuracy:            | 79.46% |
| Accuracy under attack:        | 2.25%  |
| Attack success rate:          | 97.17% |
| Average perturbed word %:     | 16.62% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 69.96  |
+-------------------------------+--------+


In [20]:
runAttack(disbert_model_wrapper_rf_mr, -1, 'attack_results_new/disbertmrattackrf04.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattackrf04.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 0 / 1 / 5:   0%|                         | 5/1066 [00:27<1:38:34,  5.57s/it]textattack: Saving checkpoint under "checkpoints\1720040675567.ta.chkpt" at 2024-07-04 05:04:35 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 0 / 2 / 6:   1%|▏                        | 6/1066 [00:27<1:22:15,  4.66s/it]

[Succeeded / Failed / Skipped / Total] 7 / 1 / 2 / 10:   1%|▏                      | 10/1066 [00:43<1:16:36,  4.35s/it]textattack: Saving checkpoint under "checkpoints\1720040691223.ta.chkpt" at 2024-07-04 05:04:51 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 2 / 3 / 15:   1%|▎                     | 15/1066 [01:08<1:20:02,  4.57s/it]textattack: Saving checkpoint under "checkpoints\1720040716231.ta.chkpt" at 2024-07-04 05:05:16 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 736 / 73 / 246 / 1055:  99%|███████████████▊| 1055/1066 [42:56<00:26,  2.44s/it]textattack: Saving checkpoint under "checkpoints\1720043223777.ta.chkpt" at 2024-07-04 05:47:03 after 1055 attacks.


[Succeeded / Failed / Skipped / Total] 740 / 74 / 246 / 1060:  99%|███████████████▉| 1060/1066 [43:13<00:14,  2.45s/it]textattack: Saving checkpoint under "checkpoints\1720043241016.ta.chkpt" at 2024-07-04 05:47:21 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 743 / 75 / 247 / 1065: 100%|███████████████▉| 1065/1066 [43:31<00:02,  2.45s/it]textattack: Saving checkpoint under "checkpoints\1720043259621.ta.chkpt" at 2024-07-04 05:47:39 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 744 / 75 / 247 / 1066: 100%|████████████████| 1066/1066 [43:33<00:00,  2.45s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 744    |
| Number of failed attacks:     | 75     |
| Number of skipped attacks:    | 247    |
| Original accuracy:            | 76.83% |
| Accuracy under attack:        | 7.04%  |
| Attack success rate:          | 90.84% |
| Average perturbed word %:     | 17.14% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 83.02  |
+-------------------------------+--------+


In [21]:
runAttack(disbert_model_wrapper_knn_mr, -1, 'attack_results_new/disbertmrattackknn04.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattackknn04.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 5 / 0 / 0 / 5:   0%|                         | 5/1066 [00:26<1:33:05,  5.26s/it]textattack: Saving checkpoint under "checkpoints\1720043288099.ta.chkpt" at 2024-07-04 05:48:08 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 5 / 0 / 2 / 7:   1%|▏                        | 7/1066 [00:26<1:06:45,  3.78s/it]

[Succeeded / Failed / Skipped / Total] 7 / 1 / 2 / 10:   1%|▏                      | 10/1066 [00:39<1:10:15,  3.99s/it]textattack: Saving checkpoint under "checkpoints\1720043301699.ta.chkpt" at 2024-07-04 05:48:21 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 2 / 3 / 15:   1%|▎                       | 15/1066 [00:51<59:56,  3.42s/it]textattack: Saving checkpoint under "checkpoints\1720043313107.ta.chkpt" at 2024-07-04 05:48:33 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 679 / 91 / 285 / 1055:  99%|███████████████▊| 1055/1066 [39:45<00:24,  2.26s/it]textattack: Saving checkpoint under "checkpoints\1720045647587.ta.chkpt" at 2024-07-04 06:27:27 after 1055 attacks.


[Succeeded / Failed / Skipped / Total] 682 / 92 / 286 / 1060:  99%|███████████████▉| 1060/1066 [40:00<00:13,  2.26s/it]textattack: Saving checkpoint under "checkpoints\1720045661908.ta.chkpt" at 2024-07-04 06:27:41 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 685 / 92 / 288 / 1065: 100%|███████████████▉| 1065/1066 [40:09<00:02,  2.26s/it]textattack: Saving checkpoint under "checkpoints\1720045671351.ta.chkpt" at 2024-07-04 06:27:51 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 686 / 92 / 288 / 1066: 100%|████████████████| 1066/1066 [40:10<00:00,  2.26s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 686    |
| Number of failed attacks:     | 92     |
| Number of skipped attacks:    | 288    |
| Original accuracy:            | 72.98% |
| Accuracy under attack:        | 8.63%  |
| Attack success rate:          | 88.17% |
| Average perturbed word %:     | 11.18% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 76.0   |
+-------------------------------+--------+


In [22]:
runAttack(disbert_model_wrapper_svc_mr, -1, 'attack_results_new/disbertmrattacksvc04.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattacksvc04.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 0 / 1 / 5:   0%|                         | 5/1066 [00:27<1:36:24,  5.45s/it]textattack: Saving checkpoint under "checkpoints\1720045699268.ta.chkpt" at 2024-07-04 06:28:19 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 0 / 3 / 7:   1%|▏                        | 7/1066 [00:27<1:09:12,  3.92s/it]

[Succeeded / Failed / Skipped / Total] 7 / 0 / 3 / 10:   1%|▏                      | 10/1066 [00:37<1:05:33,  3.72s/it]textattack: Saving checkpoint under "checkpoints\1720045709252.ta.chkpt" at 2024-07-04 06:28:29 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 0 / 4 / 15:   1%|▎                     | 15/1066 [00:56<1:05:34,  3.74s/it]textattack: Saving checkpoint under "checkpoints\1720045728154.ta.chkpt" at 2024-07-04 06:28:48 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 802 / 27 / 226 / 1055:  99%|███████████████▊| 1055/1066 [37:46<00:23,  2.15s/it]textattack: Saving checkpoint under "checkpoints\1720047938248.ta.chkpt" at 2024-07-04 07:05:38 after 1055 attacks.
[Succeeded / Failed / Skipped / Total] 802 / 27 / 227 / 1056:  99%|███████████████▊| 1056/1066 [37:46<00:21,  2.15s/it]

[Succeeded / Failed / Skipped / Total] 806 / 27 / 227 / 1060:  99%|███████████████▉| 1060/1066 [37:57<00:12,  2.15s/it]textattack: Saving checkpoint under "checkpoints\1720047949838.ta.chkpt" at 2024-07-04 07:05:49 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 810 / 27 / 228 / 1065: 100%|███████████████▉| 1065/1066 [38:13<00:02,  2.15s/it]textattack: Saving checkpoint under "checkpoints\1720047965108.ta.chkpt" at 2024-07-04 07:06:05 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 811 / 27 / 228 / 1066: 100%|████████████████| 1066/1066 [38:14<00:00,  2.15s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 811    |
| Number of failed attacks:     | 27     |
| Number of skipped attacks:    | 228    |
| Original accuracy:            | 78.61% |
| Accuracy under attack:        | 2.53%  |
| Attack success rate:          | 96.78% |
| Average perturbed word %:     | 16.61% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 70.4   |
+-------------------------------+--------+


### CLIP

In [23]:
# Load CLIP model and processor
model_name = "openai/clip-vit-base-patch32"
clip_processor = CLIPProcessor.from_pretrained(model_name)
clip_model = CLIPModel.from_pretrained(model_name)

# Access text encoder
text_encoder = clip_model.text_model

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [24]:
clipmrtrain = pd.read_csv('dataset_embeddings/clipmrtrain.txt', header=None)
clipmrtest = pd.read_csv('dataset_embeddings/clipmrtest.txt', header=None)
lr_clip_mr, rf_clip_mr, knn_clip_mr, svc_clip_mr = adaptation(clipmrtrain, clipmrtest)

Log Reg 0.7495309568480301
RF 0.7054409005628518
KNN 0.6885553470919324
SVC 0.7551594746716698


In [25]:
# custom model wrapper for CLIP
class CLIPModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).pooler_output
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [26]:
clip_model_wrapper_lr_mr = CLIPModelWrapper(clip_processor, text_encoder, lr_clip_mr)
clip_model_wrapper_rf_mr = CLIPModelWrapper(clip_processor, text_encoder, rf_clip_mr)
clip_model_wrapper_knn_mr = CLIPModelWrapper(clip_processor, text_encoder, knn_clip_mr)
clip_model_wrapper_svc_mr = CLIPModelWrapper(clip_processor, text_encoder, svc_clip_mr)

In [27]:
runAttack(clip_model_wrapper_lr_mr, -1, 'attack_results_new/clipmrattacklr04.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattacklr04.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 5 / 0 / 0 / 5:   0%|                         | 5/1066 [00:36<2:08:31,  7.27s/it]textattack: Saving checkpoint under "checkpoints\1720048706817.ta.chkpt" at 2024-07-04 07:18:26 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 5 / 0 / 1 / 6:   1%|▏                        | 6/1066 [00:36<1:47:19,  6.07s/it]

[Succeeded / Failed / Skipped / Total] 9 / 0 / 1 / 10:   1%|▏                      | 10/1066 [00:47<1:23:20,  4.74s/it]textattack: Saving checkpoint under "checkpoints\1720048717848.ta.chkpt" at 2024-07-04 07:18:37 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 14 / 0 / 1 / 15:   1%|▎                     | 15/1066 [01:15<1:28:10,  5.03s/it]textattack: Saving checkpoint under "checkpoints\1720048745988.ta.chkpt" at 2024-07-04 07:19:05 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 780 / 10 / 265 / 1055:  99%|███████████████▊| 1055/1066 [31:49<00:19,  1.81s/it]textattack: Saving checkpoint under "checkpoints\1720050580083.ta.chkpt" at 2024-07-04 07:49:40 after 1055 attacks.


[Succeeded / Failed / Skipped / Total] 784 / 10 / 266 / 1060:  99%|███████████████▉| 1060/1066 [31:58<00:10,  1.81s/it]textattack: Saving checkpoint under "checkpoints\1720050589107.ta.chkpt" at 2024-07-04 07:49:49 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 788 / 10 / 267 / 1065: 100%|███████████████▉| 1065/1066 [32:10<00:01,  1.81s/it]textattack: Saving checkpoint under "checkpoints\1720050601254.ta.chkpt" at 2024-07-04 07:50:01 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 789 / 10 / 267 / 1066: 100%|████████████████| 1066/1066 [32:11<00:00,  1.81s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 789    |
| Number of failed attacks:     | 10     |
| Number of skipped attacks:    | 267    |
| Original accuracy:            | 74.95% |
| Accuracy under attack:        | 0.94%  |
| Attack success rate:          | 98.75% |
| Average perturbed word %:     | 14.45% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 61.25  |
+-------------------------------+--------+


In [28]:
runAttack(clip_model_wrapper_rf_mr, -1, 'attack_results_new/clipmrattackrf04.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattackrf04.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 1 / 0 / 5:   0%|                       | 5/1066 [10:17<36:23:58, 123.51s/it]textattack: Saving checkpoint under "checkpoints\1720051219834.ta.chkpt" at 2024-07-04 08:00:19 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 1 / 1 / 6:   1%|▏                      | 6/1066 [10:17<30:18:41, 102.94s/it]

[Succeeded / Failed / Skipped / Total] 7 / 1 / 2 / 10:   1%|▏                     | 10/1066 [10:33<18:34:16, 63.31s/it]textattack: Saving checkpoint under "checkpoints\1720051235422.ta.chkpt" at 2024-07-04 08:00:35 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 1 / 3 / 15:   1%|▎                    | 15/1066 [10:48<12:36:45, 43.20s/it]textattack: Saving checkpoint under "checkpoints\1720051250344.ta.chkpt" at 2024-07-04 08:00:50 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 687 / 56 / 312 / 1055:  99%|███████████████▊| 1055/1066 [48:30<00:30,  2.76s/it]textattack: Saving checkpoint under "checkpoints\1720053512639.ta.chkpt" at 2024-07-04 08:38:32 after 1055 attacks.
[Succeeded / Failed / Skipped / Total] 687 / 56 / 313 / 1056:  99%|███████████████▊| 1056/1066 [48:30<00:27,  2.76s/it]

[Succeeded / Failed / Skipped / Total] 691 / 56 / 313 / 1060:  99%|███████████████▉| 1060/1066 [48:37<00:16,  2.75s/it]textattack: Saving checkpoint under "checkpoints\1720053519594.ta.chkpt" at 2024-07-04 08:38:39 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 695 / 56 / 314 / 1065: 100%|███████████████▉| 1065/1066 [48:51<00:02,  2.75s/it]textattack: Saving checkpoint under "checkpoints\1720053533863.ta.chkpt" at 2024-07-04 08:38:53 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 696 / 56 / 314 / 1066: 100%|████████████████| 1066/1066 [48:52<00:00,  2.75s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 696    |
| Number of failed attacks:     | 56     |
| Number of skipped attacks:    | 314    |
| Original accuracy:            | 70.54% |
| Accuracy under attack:        | 5.25%  |
| Attack success rate:          | 92.55% |
| Average perturbed word %:     | 13.95% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 72.14  |
+-------------------------------+--------+


In [29]:
runAttack(clip_model_wrapper_knn_mr, -1, 'attack_results_new/clipmrattackknn04.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattackknn04.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 1 / 0 / 5:   0%|                         | 5/1066 [00:29<1:45:09,  5.95s/it]textattack: Saving checkpoint under "checkpoints\1720053564566.ta.chkpt" at 2024-07-04 08:39:24 after 5 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 2 / 2 / 10:   1%|▏                      | 10/1066 [00:44<1:18:32,  4.46s/it]textattack: Saving checkpoint under "checkpoints\1720053579457.ta.chkpt" at 2024-07-04 08:39:39 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 2 / 3 / 15:   1%|▎                     | 15/1066 [00:58<1:07:52,  3.88s/it]textattack: Saving checkpoint under "checkpoints\1720053592961.ta.chkpt" at 2024-07-04 08:39:52 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 624 / 100 / 331 / 1055:  99%|██████████████▊| 1055/1066 [41:23<00:25,  2.35s/it]textattack: Saving checkpoint under "checkpoints\1720056017936.ta.chkpt" at 2024-07-04 09:20:17 after 1055 attacks.


[Succeeded / Failed / Skipped / Total] 629 / 100 / 331 / 1060:  99%|██████████████▉| 1060/1066 [41:34<00:14,  2.35s/it]textattack: Saving checkpoint under "checkpoints\1720056029487.ta.chkpt" at 2024-07-04 09:20:29 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 633 / 100 / 332 / 1065: 100%|██████████████▉| 1065/1066 [41:41<00:02,  2.35s/it]textattack: Saving checkpoint under "checkpoints\1720056036595.ta.chkpt" at 2024-07-04 09:20:36 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 634 / 100 / 332 / 1066: 100%|███████████████| 1066/1066 [41:42<00:00,  2.35s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 634    |
| Number of failed attacks:     | 100    |
| Number of skipped attacks:    | 332    |
| Original accuracy:            | 68.86% |
| Accuracy under attack:        | 9.38%  |
| Attack success rate:          | 86.38% |
| Average perturbed word %:     | 10.35% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 75.67  |
+-------------------------------+--------+


In [30]:
runAttack(clip_model_wrapper_svc_mr, -1, 'attack_results_new/clipmrattacksvc04.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattacksvc04.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 5 / 0 / 0 / 5:   0%|                         | 5/1066 [01:07<3:59:49, 13.56s/it]textattack: Saving checkpoint under "checkpoints\1720056105301.ta.chkpt" at 2024-07-04 09:21:45 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 5 / 0 / 1 / 6:   1%|▏                        | 6/1066 [01:07<3:20:07, 11.33s/it]

[Succeeded / Failed / Skipped / Total] 8 / 0 / 2 / 10:   1%|▏                      | 10/1066 [01:16<2:13:51,  7.61s/it]textattack: Saving checkpoint under "checkpoints\1720056113541.ta.chkpt" at 2024-07-04 09:21:53 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 12 / 0 / 3 / 15:   1%|▎                     | 15/1066 [01:39<1:55:45,  6.61s/it]textattack: Saving checkpoint under "checkpoints\1720056136615.ta.chkpt" at 2024-07-04 09:22:16 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 782 / 12 / 261 / 1055:  99%|███████████████▊| 1055/1066 [35:37<00:22,  2.03s/it]textattack: Saving checkpoint under "checkpoints\1720058174784.ta.chkpt" at 2024-07-04 09:56:14 after 1055 attacks.
[Succeeded / Failed / Skipped / Total] 782 / 12 / 262 / 1056:  99%|███████████████▊| 1056/1066 [35:37<00:20,  2.02s/it]

[Succeeded / Failed / Skipped / Total] 786 / 12 / 262 / 1060:  99%|███████████████▉| 1060/1066 [35:46<00:12,  2.02s/it]textattack: Saving checkpoint under "checkpoints\1720058183867.ta.chkpt" at 2024-07-04 09:56:23 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 790 / 12 / 263 / 1065: 100%|███████████████▉| 1065/1066 [35:59<00:02,  2.03s/it]textattack: Saving checkpoint under "checkpoints\1720058197095.ta.chkpt" at 2024-07-04 09:56:37 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 791 / 12 / 263 / 1066: 100%|████████████████| 1066/1066 [36:00<00:00,  2.03s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 791    |
| Number of failed attacks:     | 12     |
| Number of skipped attacks:    | 263    |
| Original accuracy:            | 75.33% |
| Accuracy under attack:        | 1.13%  |
| Attack success rate:          | 98.51% |
| Average perturbed word %:     | 14.33% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 61.21  |
+-------------------------------+--------+
